In [17]:
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
import pandas as pd
import numpy as np
import altair as alt
import project_functions2 as pf

from sklearn.linear_model import LinearRegression
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [18]:
alt.renderers.enable('default')
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [19]:
googl = yf.Ticker('GOOGL')
amzn = yf.Ticker('AMZN')
aapl = yf.Ticker('AAPL')
fb = yf.Ticker('FB')
msft = yf.Ticker('MSFT')
tsla = yf.Ticker('TSLA')
stock_list = ['AMZN', 'AAPL', 'FB','GOOGL', 'MSFT', 'TSLA']

In [20]:
aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')
stock_objects = {}
for stock in stock_list:
    stock_objects[stock] = yf.Ticker(stock)

In [21]:
# This is the same prep that needs to be done to each stock as seen in the single stock
# predictor. This HAS to be done prior to merging the dataframes
stock_dfs = [aapl_df, amzn_df, fb_df, googl_df, msft_df, tsla_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_percent_change_setup(stock_df, 5)

In [22]:
# putting the stock dfs in a list for the cobiner function


In [23]:
def combiner(stock_dfs):
    combine_df = stock_dfs[0]
    i = 1
    while i < len(stock_dfs):
        combine_df = pd.concat([combine_df, stock_dfs[i]])
        i += 1
    
    combine_df.sort_values(by=['Date'], inplace=True)
    return combine_df

In [24]:
combine_df = combiner(stock_dfs)
combine_df

Open         High          Low        Close     Volume  \
Date                                                                        
1980-12-12     0.100922     0.101361     0.100922     0.100922  469033600   
1980-12-15     0.096096     0.096096     0.095657     0.095657  175884800   
1980-12-16     0.089075     0.089075     0.088636     0.088636  105728000   
1980-12-17     0.090830     0.091268     0.090830     0.090830   86441600   
1980-12-18     0.093463     0.093902     0.093463     0.093463   73449600   
...                 ...          ...          ...          ...        ...   
2021-03-25   235.300003   236.930099   231.570007   232.339996   32779842   
2021-03-25  2029.050049  2044.540039  1996.089966  2032.459961    1749694   
2021-03-25  3151.040039  3109.780029  3037.139893  3046.260010    3474181   
2021-03-25   280.980011   288.059998   277.750000   278.739990   23539057   
2021-03-25   613.000000   645.460022   609.969971   640.390015   38858703   

            Dividends  Stock Splits  Day  Month  Year  ...  10 Day RSI  \
Date                                                   ...               
1980-12-12        0.0           0.0   12     12  1980  ...    9.167035   
1980-12-15        0.0           0.0   15     12  1980  ...    2.379573   
1980-12-16        0.0           0.0   16     12  1980  ...   -0.828352   
1980-12-17        0.0           0.0   17     12  1980  ...    0.002617   
1980-12-18        0.0           0.0   18     12  1980  ...    0.578574   
...               ...           ...  ...    ...   ...  ...         ...   
2021-03-25        0.0           0.0   25      3  2021  ...   -0.195309   
2021-03-25        0.0           0.0   25      3  2021  ...   -0.321878   
2021-03-25        0.0           0.0   25      3  2021  ...   -0.207201   
2021-03-25        0.0           0.0   25      3  2021  ...    0.199289   
2021-03-25        0.0           0.0   25      3  2021  ...   -0.830382   

            20 Day RSI  Larry Williams R 5 Day  Larry Williams R 10 Day  \
Date                                                                      
1980-12-12    9.167035              100.000000               100.000000   
1980-12-15    2.379573              100.000000               100.000000   
1980-12-16   -0.828352              100.000000               100.000000   
1980-12-17    0.002617               82.759554                82.759554   
1980-12-18    0.578574               62.065000                62.065000   
...                ...                     ...                      ...   
2021-03-25    0.085925               74.444521                74.444521   
2021-03-25    0.058639               49.140081                69.070494   
2021-03-25   -0.002757               82.082539                82.082539   
2021-03-25    0.476484               95.491846                58.755953   
2021-03-25   -0.155470               66.068003                70.526076   

            Larry Williams R 20 Day  AD Oscillator  5 Day CCI  10 Day CCI  \
Date                                                                        
1980-12-12               100.000000       1.000000   0.000000    0.000000   
1980-12-15               100.000000       1.000000 -66.666667  -66.666667   
1980-12-16               100.000000       1.000000 -85.249616  -85.249616   
1980-12-17                82.759554       1.000000 -44.842205  -44.842205   
1980-12-18                62.065000       1.000000  -6.900048   -6.900048   
...                             ...            ...        ...         ...   
2021-03-25                51.876130       0.856348 -27.609706  -34.142946   
2021-03-25                67.880438       0.249330 -62.715563  -74.218348   
2021-03-25                45.096342       0.874448 -63.190808  -58.104773   
2021-03-25                45.379799       0.903978 -97.539299   -5.998718   
2021-03-25                44.444430       0.142857 -82.427307 -114.431915   

            20 Day CCI  Percentage Change in 5 Days  
Date                  

In [25]:
def combo_multi_linear_regressor(combine_df, split_time, stock_dfs):
    
    X = combine_df.iloc[:,:-1]
    y = combine_df.iloc[:,-1:]
    #X = stock_df['Days From IPO'].values.reshape(-1, 1)
    #y = stock_df['Close'].values.reshape(-1, 1)
    
    # Does train/Test Split on chosen time
    # Change the -50 to a differnt value to change split point
    split_mark = int(len(combine_df)-(split_time*len(stock_dfs)))
    X_train = X.head(split_mark)
    X_test = X.tail(len(combine_df) - split_mark)
    y_train = y.head(split_mark)
    y_test = y.tail(len(combine_df) - split_mark)
    
    stock_model = LinearRegression().fit(X_train, y_train)
    stock_close_pred = stock_model.predict(X_test)
    stock_train_pred = stock_model.predict(X_train)
    
    columns = ['Apple Price', 'Apple Prediction', 
               'Amazon Price', 'Amazon Prediction',
               'Facebook Price', 'Facebook Prediction',
               'Google Price', 'Google Prediction',
               'Microsoft Price', 'Microsoft Prediction',
               'Tesla Price', 'Tesla Prediction']
    
    single_split_mark = int(len(stock_dfs[0])-split_time)
    results_df = pd.DataFrame(columns=columns, index=stock_dfs[0].tail(len(stock_dfs[0])-single_split_mark).index)

    i = 0
    for stock_df in stock_dfs:
        single_split_mark = int(len(stock_df)-split_time)
        results_df[columns[i]] = stock_df.iloc[:,-1:].tail(len(stock_df) - single_split_mark)
        results_df[columns[i+1]] = stock_model.predict(stock_df.iloc[:,:-1].tail(len(stock_df) - single_split_mark))
        i += 2
        
    results_df.reset_index(inplace=True)
    results_df = results_df.melt('Date', var_name='Company', value_name='Price')
    # Plots Results
    line_plot = alt.Chart(results_df).mark_line().encode(
        x = 'Date',
        y = 'Price',
        color = 'Company'
    )
    
    
    
    train_score = r2_score(y_train, stock_train_pred)
    print("Training R2 Score: " + str(train_score))
    print()

    
    
    model_score = r2_score(y_test.dropna(), stock_close_pred[:len(y_test.dropna())])
    print()
    print("R2 Score: " + str(model_score))
    
    return line_plot

In [15]:
aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')

stock_dfs = [aapl_df, amzn_df, fb_df, googl_df, msft_df, tsla_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_percent_change_setup(stock_df, 5)
    
combine_df = combiner(stock_dfs)

combo_multi_linear_regressor(combine_df, 365, stock_dfs)

Training R2 Score: 0.003124696253698378


R2 Score: -0.09722962396300039


alt.Chart(...)

In [36]:
def neural_net(combine_df, split_time, stock_dfs):
    scaler = MinMaxScaler()
    X = combine_df.iloc[:,:-1]
    y = combine_df.iloc[:,-1:]
    #X = stock_df['Days From IPO'].values.reshape(-1, 1)
    #y = stock_df['Close'].values.reshape(-1, 1)
    
    # Does train/Test Split on last year
    # Change the -50 to a differnt value to change split point
    split_mark = int(len(combine_df)-(split_time*len(stock_dfs)))
    X_train = X.head(split_mark)
    X_test = X.tail(len(combine_df) - split_mark)
    y_train = y.head(split_mark)
    y_test = y.tail(len(combine_df) - split_mark)

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    stock_nn = MLPRegressor(activation='relu', hidden_layer_sizes = [500, 500, 500], random_state=45).fit(X_train_scaled, y_train)
    
    stock_nn_pred = stock_nn.predict(X_test_scaled)
    stock_nn_train_pred = stock_nn.predict(X_train_scaled)
    
    sell_line = stock_nn_pred*1.1
    buy_line = stock_nn_pred*0.9
    
    # Plots Results

    train_score = r2_score(y_train, stock_nn_train_pred)
    print("Training R2 Score: " + str(train_score))
    print()

    columns = ['Apple Price', 'Apple Prediction', 
               'Amazon Price', 'Amazon Prediction',
               'Facebook Price', 'Facebook Prediction',
               'Google Price', 'Google Prediction',
               'Microsoft Price', 'Microsoft Prediction',
               'Tesla Price', 'Tesla Prediction']
    
    single_split_mark = int(len(stock_dfs[0])-split_time)
    results_df = pd.DataFrame(columns=columns, index=stock_dfs[0].tail(len(stock_dfs[0])-single_split_mark).index)

    i = 0
    for stock_df in stock_dfs:
        single_split_mark = int(len(stock_df)-split_time)
        results_df[columns[i]] = stock_df.iloc[:,-1:].tail(len(stock_df) - single_split_mark)
        curr_X_train = scaler.transform(stock_df.iloc[:,:-1].tail(len(stock_df) - single_split_mark))
        results_df[columns[i+1]] = stock_nn.predict(curr_X_train)
        i += 2
        
    results_df.reset_index(inplace=True)
    results_df = results_df.melt('Date', var_name='Company', value_name='Price')
    # Plots Results
    line_plot = alt.Chart(results_df).mark_line().encode(
        x = 'Date',
        y = 'Price',
        color = 'Company'
    )
    
    model_score = r2_score(y_test.dropna(), stock_nn_pred[:len(y_test.dropna())])
    print("R2 Score: " + str(model_score))
    
    return line_plot

In [37]:
def neural_net_y_scaled(combine_df, split_time, stock_dfs):
    scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    X = combine_df.iloc[:,:-1]
    y = combine_df.iloc[:,-1:]
    #X = stock_df['Days From IPO'].values.reshape(-1, 1)
    #y = stock_df['Close'].values.reshape(-1, 1)
    
    # Does train/Test Split on last year
    # Change the -50 to a differnt value to change split point
    split_mark = int(len(combine_df)-(split_time*len(stock_dfs)))
    X_train = X.head(split_mark)
    X_test = X.tail(len(combine_df) - split_mark)
    y_train = y.head(split_mark)
    y_test = y.tail(len(combine_df) - split_mark)

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    y_train_scaled = y_scaler.fit_transform(y_train)
    y_test_scaled = y_scaler.transform(y_test)
    
    stock_nn = MLPRegressor(activation='relu', hidden_layer_sizes = [500, 500, 500], random_state=45).fit(X_train_scaled, y_train_scaled)
    
    stock_nn_pred_pre_scale = stock_nn.predict(X_test_scaled)
    stock_nn_train_pred_pre_scale = stock_nn.predict(X_train_scaled)
    
    stock_nn_pred = y_scaler.inverse_transform(stock_nn_pred_pre_scale.reshape(-1, 1))
    stock_nn_train_pred = y_scaler.inverse_transform(stock_nn_train_pred_pre_scale.reshape(-1, 1))
    
    #sell_line = stock_nn_pred*1.1
    #buy_line = stock_nn_pred*0.9
    
    # Plots Results

    train_score = r2_score(y_train, stock_nn_train_pred)
    print("Training R2 Score: " + str(train_score))
    print()

    columns = ['Apple Price', 'Apple Prediction', 
               'Amazon Price', 'Amazon Prediction',
               'Facebook Price', 'Facebook Prediction',
               'Google Price', 'Google Prediction',
               'Microsoft Price', 'Microsoft Prediction',
               'Tesla Price', 'Tesla Prediction']
    
    single_split_mark = int(len(stock_dfs[0])-split_time)
    results_df = pd.DataFrame(columns=columns, index=stock_dfs[0].tail(len(stock_dfs[0])-single_split_mark).index)

    i = 0
    for stock_df in stock_dfs:
        single_split_mark = int(len(stock_df)-split_time)
        results_df[columns[i]] = stock_df.iloc[:,-1:].tail(len(stock_df) - single_split_mark)
        curr_X_train = scaler.transform(stock_df.iloc[:,:-1].tail(len(stock_df) - single_split_mark))
        results_df[columns[i+1]] = stock_nn.predict(curr_X_train)
        i += 2
        
    results_df.reset_index(inplace=True)
    results_df = results_df.melt('Date', var_name='Company', value_name='Price')
    # Plots Results
    line_plot = alt.Chart(results_df).mark_line().encode(
        x = 'Date',
        y = 'Price',
        color = 'Company'
    )
    
    model_score = r2_score(y_test.dropna(), stock_nn_pred[:len(y_test.dropna())])
    print("R2 Score: " + str(model_score))
    
    return line_plot

In [17]:
aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')

stock_dfs = [aapl_df, amzn_df, fb_df, googl_df, msft_df, tsla_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_percent_change_setup(stock_df, 5)
    
combine_df = combiner(stock_dfs)

neural_net(combine_df, 365, stock_dfs)

Training R2 Score: -0.005519120989097237

R2 Score: -0.1424805630264745


alt.Chart(...)

In [18]:
aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')

stock_dfs = [aapl_df, amzn_df, fb_df, googl_df, msft_df, tsla_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_percent_change_setup(stock_df, 20)
    
combo_multi_linear_regressor(combine_df, 365, stock_dfs)

Training R2 Score: 0.003124696253698378


R2 Score: -0.09722962396300039


alt.Chart(...)

In [19]:
aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')

stock_dfs = [aapl_df, amzn_df, fb_df, googl_df, msft_df, tsla_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_percent_change_setup(stock_df, 1)
    
combine_df = combiner(stock_dfs)

combo_multi_linear_regressor(combine_df, 365, stock_dfs)

Training R2 Score: 0.001040271015227301


R2 Score: -0.015467827438397919


alt.Chart(...)

In [20]:
aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')

stock_dfs = [aapl_df, amzn_df, fb_df, googl_df, msft_df, tsla_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_percent_change_setup(stock_df, 1)
    
combine_df = combiner(stock_dfs)

neural_net(combine_df, 365, stock_dfs)

Training R2 Score: -0.024273963881713767

R2 Score: -0.06516512409139152


alt.Chart(...)

In [25]:
combine_df, stock_dfs_dict = pf.end_to_end_lstm_prep(stock_objects, lookback=5, pred_len=5)
stock_dfs = []
for stock in stock_dfs_dict:
    stock_dfs.append(stock_dfs_dict[stock])
combo_multi_linear_regressor(combine_df, 365, stock_dfs)

Training R2 Score: 0.7158711858450173


R2 Score: 0.176370641607547


alt.Chart(...)

In [14]:
aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')

stock_dfs = [aapl_df, amzn_df, fb_df, googl_df, msft_df, tsla_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_close_setup(stock_df, 5)
    
combine_df = combiner(stock_dfs)

neural_net(combine_df, 365, stock_dfs)

Training R2 Score: 0.998211843884632

R2 Score: 0.9957630976201084


alt.Chart(...)

In [15]:
aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')

stock_dfs = [aapl_df, amzn_df, fb_df, googl_df, msft_df, tsla_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_close_setup(stock_df, 5)
    
combine_df = combiner(stock_dfs)
combo_multi_linear_regressor(combine_df, 365, stock_dfs)

Training R2 Score: 0.9979308962425268


R2 Score: 0.9940529997290041


alt.Chart(...)

In [38]:
aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')

stock_dfs = [aapl_df, amzn_df, fb_df, googl_df, msft_df, tsla_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_close_setup(stock_df, 5)
    
combine_df = combiner(stock_dfs)

neural_net(combine_df, 365, stock_dfs)

Training R2 Score: 0.9955501903692586

R2 Score: 0.9926202598442966


alt.Chart(...)

In [39]:
aapl_df = aapl.history(period='max')
amzn_df = amzn.history(period='max')
fb_df = fb.history(period='max')
googl_df = googl.history(period='max')
msft_df = msft.history(period='max')
tsla_df = tsla.history(period='max')

stock_dfs = [aapl_df, amzn_df, fb_df, googl_df, msft_df, tsla_df]

for stock_df in stock_dfs:
    stock_df = pf.date_time_prep(stock_df) 
    stock_df = pf.rolling_aves(stock_df)
    stock_df = pf.future_close_setup(stock_df, 5)
    
combine_df = combiner(stock_dfs)

neural_net_y_scaled(combine_df, 365, stock_dfs)

Training R2 Score: 0.9983672001630034

R2 Score: 0.9944135057295579


alt.Chart(...)